In [ ]:
from airflow.decorators import dag, task
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.providers.postgres.operators.postgres import PostgresOperator


In [ ]:
@dag(dag_id = 'sythetic_data_load_postgres',
     start_date=datetime(2023,5,20),
    )
def load_synthetic_data():
    @task
    def create_table():
        create_employees_table = PostgresOperator(
        task_id="create_cc_table",
        postgres_conn_id="tutorial_pg_conn",
        sql="""
            CREATE TABLE IF NOT EXISTS cc_details (
                "cc_num" TEXT,
                "cust_num" TEXT,
                "credit_limit" INTEGER,
                "zip_code" TEXT,
                "cash_back_pct" FLOAT,
                "ftr_dttm" text
            );""",
        )
    @task
    def load_data():
        data_path = '/opt/airflow/data/cc_data.csv'
        postgres_hook = PostgresHook(postgres_conn_id="tutorial_pg_conn")
        conn = postgres_hook.get_conn()
        cur = conn.cursor()
        with open(data_path, "r") as file:
            cur.copy_expert(
                "COPY cc_details FROM STDIN WITH CSV HEADER DELIMITER AS ',' ",
                file,
            )
        conn.commit()
    create_table() >> load_data()

dag = load_synthetic_data()